# Example reading PALEO netcdf output using python xarray

Generate file 'test.nc' with:

    julia> include("PALEO_examples_shelf1D_P_O2.jl")
    julia> PALEOmodel.OutputWriters.save_netcdf(paleorun.output, "test.nc")

In [ ]:
import os

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
os.getcwd()

In [ ]:
# read top-level dataset
ds = xr.open_dataset("test.nc")
ds.attrs

In [ ]:
# read ocean group (= PALEO Domain ocean) from netcdf file
ds_ocean = xr.open_dataset("test.nc", group="ocean")

# attach z coordinates (this is not automatic!)
ds_ocean = ds_ocean.set_coords(["zmid", "zlower", "zupper"])

In [ ]:
ds_ocean

In [ ]:
# timeseries for a scalar variable
print(ds_ocean["O2_total"])
ds_ocean["O2_total"].plot()

In [ ]:
# split dataset into columns (1 column only here so this could be skipped)
cells_in_column = ds_ocean["cells_in_column"]

print("Icolumn comment:", ds_ocean["Icolumns"].attrs["comment"]) # zero-based indices of cells from top to bottom ordered by columns
Icolumns = ds_ocean["Icolumns"].values

ds_ocean_columns = []
colstartidx = 0

for cidx, clength in enumerate(cells_in_column.values):
    ccells = Icolumns[np.arange(colstartidx, colstartidx + clength)]
    ds_ocean_columns.append(ds_ocean.isel(cells=ccells, columns=cidx))
    colstartidx += clength
    


In [ ]:
ds_ocean_columns[0]

In [ ]:
ds_ocean_columns[0].columnnames.values

In [ ]:
print(ds_ocean_columns[0]["P_conc"])

In [ ]:
ds_ocean_columns[0]["P_conc"].dims

In [ ]:
ds_ocean_columns[0]["P_conc"].coords

In [ ]:
np.shape(ds_ocean_columns[0]["P_conc"].values)

In [ ]:
ds_ocean_columns[0]["P_conc"].sel(tmodel=2.5).plot(y="zmid")